# Decorators

In this tutorial on decorators, we’ll look at what they are and how to create and use them. Decorators provide a simple syntax for calling **higher-order functions**.

By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

---

## Functions

Before you can understand decorators, you must first understand how functions work. For our purposes, **a function returns a value based on the given arguments.** Here is a very simple example:

in general, functions in Python may also have side effects rather than just turning an input into an output. The print() function is a basic example of this: it ``returns None`` while having the side effect of outputting something to the console.

> **Note:** In **functional programming**, you work (almost) only with pure functions without side effects. While not a purely functional language, Python supports many of the functional programming concepts, including functions as first-class objects.

## First-Class Objects

In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (string, int, float, list, and so on).

In [14]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

In [15]:
greet_bob(say_hello)

'Hello Bob'

Note that ``greet_bob(say_hello)`` refers to two functions, but in different ways: ``greet_bob()`` and say_hello. The say_hello function is named without parentheses. This means that only a reference to the function is passed. The function is not executed. The ``greet_bob()`` function, on the other hand, is written with parentheses, so it will be called as usual.

## Inner Functions

It’s possible to define functions inside other functions. Such functions are called inner functions. Here’s an example of a function with two inner functions:

In [26]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

In [27]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


In [28]:
first_child()

NameError: name 'first_child' is not defined

Whenever you call parent(), the inner functions first_child() and second_child() are also called. But because of their local scope, they aren’t available outside of the parent() function.

## Returning Functions From Functions

Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer parent() function:

In [32]:
def parent(num):
    def first_child():
        return "Hi, I am Rasool"

    def second_child():
        return "Call me RA"

    if num == 1:
        return first_child
    else:
        return second_child

Note that you are returning first_child without the parentheses. Recall that this means that you are returning a reference to the function first_child. In contrast first_child() with parentheses refers to the result of evaluating the function. This can be seen in the following example:

In [43]:
first =  parent(1)
second = parent(2)

In [44]:
first

<function __main__.parent.<locals>.first_child()>

In [39]:
second

<function __main__.parent.<locals>.second_child()>

In [40]:
first()

'Hi, I am Emma'

## Simple Decorators

Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [47]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [48]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

> **decorators wrap a function, modifying its behavior.** 

## Syntactic Sugar

The way you decorated say_whee() above is a little clunky. First of all, you end up typing the name say_whee three times. In addition, the decoration gets a bit hidden away below the definition of the function.

Instead, Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the **“pie” syntax.** The following example does the exact same thing as the first decorator example:

In [53]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

> So, @my_decorator is just an easier way of saying say_whee = my_decorator(say_whee). It’s how you apply a decorator to a function.

## Decorating Functions With Arguments

Say that you have a function that accepts some arguments. Can you still decorate it? Let’s try:



In [64]:
def do_twice(func):
    def wraper():
        func()
        func()
    return wraper
        

@do_twice
def greet(name):
    print(f"Hello {name}")

In [66]:
greet("world")

TypeError: do_twice.<locals>.wraper() takes 0 positional arguments but 1 was given

The problem is that the inner function wrapper_do_twice() does not take any arguments, but name="World" was passed to it. You could fix this by letting wrapper_do_twice() accept one argument, but then it would not work for the say_whee() function you created earlier.

The solution is to use ``*args and **kwargs`` in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite decorators.py as follows:

In [68]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [69]:
@do_twice
def greet(name):
    print(f"Hello {name}")

In [71]:
greet("world")

Hello world
Hello world


## Returning Values From Decorated Functions

What happens to the return value of decorated functions? Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:

In [74]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [77]:
res = return_greeting("world")

Creating greeting
Creating greeting


In [78]:
print(res)

None


Oops, your decorator ate the return value from the function.

Because ``wrapper()`` doesn’t explicitly return a value, the call return_greeting("world") ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function.

In [80]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [81]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [82]:
res = return_greeting("Adam")

Creating greeting
Creating greeting


In [83]:
res

'Hi Adam'

## Who Are You, Really?

A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability. Introspection is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and documentation:

In [86]:
print

<function print>

In [87]:
print.__name__

'print'

In [88]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [89]:
return_greeting

<function __main__.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>

To fix this, decorators should use the @functools.wraps decorator, which will preserve information about the original function. 

In [91]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [94]:
@do_twice
def return_greeting(name):
    """
    this function says hi to user
    """
    print("Creating greeting")
    return f"Hi {name}"

In [95]:
help(return_greeting)

Help on function return_greeting in module __main__:

return_greeting(name)
    this function says hi to user



## A Real World Example

Let’s look at a few more useful examples of decorators. You’ll notice that they’ll mainly follow the same pattern that you’ve learned so far:

In [97]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

## Timing Functions

In [101]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [102]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0062 secs


In [103]:
waste_some_time(100)

Finished 'waste_some_time' in 0.3982 secs
